# Before running the following client side program: 

- first run "eva_server" in terminal with you python venv(i.e. test_eva_db you just created). 

- run this ipynb under the same python venv as that of server

In [ ]:
!wget -nc "https://raw.githubusercontent.com/sashiko-345/eva/master/dog.jpg"
!wget -nc https://www.dropbox.com/s/k00wge9exwkfxz6/ua_detrac.mp4
!wget "https://raw.githubusercontent.com/sashiko-345/eva/master/eva/udfs/camera_trap_animal_detector.py" 
!mv 'camera_trap_animal_detector.py' 'downloaded_camera_trap_animal_detector.py'

!pip install timm


In [2]:
# first run "eva_server" in terminal with you python venv(i.e. test_eva_db you just created)

# run this ipynb under the same python venv as that of server

# connect to server
from eva.server.db_api import connect
import nest_asyncio
nest_asyncio.apply()
connection = connect(host = '127.0.0.1', port = 5435)
cursor = connection.cursor()

In [8]:
# load
cursor.execute('LOAD FILE "dog.jpg" INTO singlePic;')
response = cursor.fetch_all()
print(response)

@status: -1
@batch: None
@error: Failed to find a video file at location: dog.jpg


In [ ]:
# load
cursor.execute('LOAD FILE "ua_detrac.mp4" INTO singleVid;')
response = cursor.fetch_all()
print(response)

In [3]:
drop_udf_query = """DROP UDF AnimalDetector;
        """
cursor.execute(drop_udf_query)
response = cursor.fetch_all()
print(response)

@status: 0
@batch: Batch Object:
@dataframe:                                          0
0  UDF AnimalDetector successfully dropped
@batch_size: 1
@identifier_column: None
@query_time: 0.01660633000000189


In [ ]:
import os
udf_path = os.getcwd()
udf_path += '/downloaded_camera_trap_animal_detector.py'
create_udf_query = f"""CREATE UDF IF NOT EXISTS AnimalDetector
                  INPUT  (frame NDARRAY UINT8(3, ANYDIM, ANYDIM))
                  OUTPUT (labels NDARRAY STR(ANYDIM),
                          scores NDARRAY FLOAT32(ANYDIM))
                  TYPE  Classification
                  IMPL  '{udf_path}';
        """
# load
cursor.execute(create_udf_query)
response = cursor.fetch_all()
print(response)

In [17]:
# query dog with animal detector
cursor.execute('''SELECT id, AnimalDetector(data) FROM singleVid WHERE id < 5 AND ['non_blank'] <@ AnimalDetector(data).labels''')
response = cursor.fetch_all()
print(response)
# where id does not work for uploaded picture, only work for videos

@status: 0
@batch: Batch Object:
@dataframe:    singlevid.id animaldetector.labels animaldetector.scores
0             0           [non_blank]  [0.9883119985000001]
1             1           [non_blank]        [0.9384851195]
2             2           [non_blank]  [0.9649098702000001]
3             3           [non_blank]        [0.9768140875]
4             4           [non_blank]  [0.9880527854000001]
5             0           [non_blank]        [0.9906347571]
6             1           [non_blank]        [0.9416899234]
7             2           [non_blank]  [0.9451785050000001]
8             3           [non_blank]  [0.9777551666000001]
9             4           [non_blank]  [0.9823591486000001]
@batch_size: 10
@identifier_column: None
@query_time: 173.39862757499986


In [ ]:
cursor.execute('''SELECT id, AnimalDetector(data) FROM singleVid WHERE id < 1''')
response = cursor.fetch_all()
print(response)